# CIFAR-10: Training and Testing on a Clean Dataset & Adversarial Analysis

## Imports and CIFAR-10 loading

In [15]:
# Imports all the module paths
import sys

sys.path.append("../")

# Loads the rest of the modules
import torch
import torch.nn as nn

# File containing all the required training methods
import defenses.cifar10 as cifar10

# For testing
import utils.clean_test as clean_test

# Contains the data loadders
import utils.dataloaders as dataloaders

# For printing outcomes
# import utils.printing as printing

# Example printing, but I removed it to simplify results
# for epsilon in epsilons:
#     printing.print_attack(
#         model,
#         testSetLoader,
#         "FGSM",
#         attacks["FGSM"],
#         epsilon=epsilon,
#     )

## Load the dataset

In [16]:
DATA_ROOT = "../datasets/CIFAR10"

trainSetLoader, _, testSetLoader = dataloaders.get_CIFAR10_data_loaders(
    DATA_ROOT,
    trainSetSize=50000,
    validationSetSize=0,
    batchSize=128,
)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


## Note: the functions also provide a way to have shorter training times. However, this provides worse results. See my thesis for why.

## Standard Training

In [17]:
SAVE_LOAD_ROOT = "../models_data/CIFAR10"

standard_model = cifar10.standard_training(
    trainSetLoader,
    load_if_available=True,
    load_path=SAVE_LOAD_ROOT + "/cifar10_standard",
)

Found already trained model...
... loaded!


In [18]:
# Test the model
clean_test.test_trained_model(standard_model, testSetLoader)

Testing the model...


Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 95.25%


In [19]:
# Save the model
torch.save(standard_model, SAVE_LOAD_ROOT + "/cifar10_standard")

## Attacks and Their Results

In [20]:
# A possible attacks array (for nice printing):
import utils.attacking as attacking

attacks = {}

### FGSM (Fast Sign Gradient Method) https://arxiv.org/abs/1412.6572

The formula that is used for producing the adversarial example in this case is:

$x' = x + \epsilon * sign(\nabla_{x}J(\theta, x, y))$, where J is the loss function used.

In [21]:
import attacks.fgsm as fgsm

attacks["FGSM"] = fgsm.fgsm_attack

In [22]:
# Several values to use for the epsilons
epsilons = [0, 0.05, 0.1, 0.2, 0.35, 0.55, 0.75, 1]

In [23]:
# Run test for each epsilon
for epsilon in epsilons:
    attacking.attack_model(
        standard_model,
        testSetLoader,
        "FGSM",
        attacks["FGSM"],
        epsilon=epsilon,
    )

Testing the model under FGSM Attack using epsilon = 0, alpha = None...


FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 95.25%


Testing the model under FGSM Attack using epsilon = 0.05, alpha = None...


FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 32.96%


Testing the model under FGSM Attack using epsilon = 0.1, alpha = None...


...

... done! Accuracy: 8.79%


Testing the model under FGSM Attack using epsilon = 0.75, alpha = None...


FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 8.93%


Testing the model under FGSM Attack using epsilon = 1, alpha = None...


FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 9.07%




### I-FGSM / BIM (Iterative FGSM / Basic Iterative Method) https://arxiv.org/abs/1607.02533

The formula that is used for producing the adversarial example in this case is:

$x'_{0} = x$
$x'_{k + 1} = Clip_{x, \epsilon} \{x'_{k} + \alpha * sign(\nabla_{x}J(\theta, x'_{k}, y)\}$, where J is the loss function used. Note that in the paper proposing this method, $\alpha = 1$ is used, so pixels are changed by 1 in each iteration.

The formula for the clip function is also given in the paper.

In [24]:
import attacks.ifgsm as ifgsm

attacks["I-FGSM"] = ifgsm.ifgsm_attack

In [25]:
# Several values for epsilon. It's interesting to see how clamping affects results
epsilons = [0, 4 / 255, 0.05, 0.1, 0.2, 0.35, 0.55]

In [26]:
for epsilon in epsilons:
    attacking.attack_model(
        standard_model,
        testSetLoader,
        "I-FGSM",
        attacks["I-FGSM"],
        epsilon=epsilon,
        alpha=1,
        iterations=3,
    )

Testing the model under I-FGSM Attack using epsilon = 0, alpha = 1...


I-FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 95.25%


Testing the model under I-FGSM Attack using epsilon = 0.01568627450980392, alpha = 1...


I-FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 1.5%


Testing the model under I-FGSM Attack using epsilon = 0.05, alpha = 1...


...

... done! Accuracy: 0.0%


Testing the model under I-FGSM Attack using epsilon = 0.35, alpha = 1...


I-FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 0.04%


Testing the model under I-FGSM Attack using epsilon = 0.55, alpha = 1...


I-FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 2.48%




### PGD (Projected Gradient Descent) https://arxiv.org/pdf/1706.06083.pdf

The formula that is used for producing the adversarial example in this case is:

$x^{t+1} = \Pi_{x+S}(x^t+\alpha sgn(\bigtriangledown_x L(\theta, x, y)))$\n
where S is a set of allowed perturbations. Note that this is basically BIM with random initialisation. 

We use the $L_{\infty}$ norm here, i.e. we use clamping. 

In [27]:
import attacks.pgd as pgd

attacks["PGD"] = pgd.pgd_attack

In [28]:
# Several values to use for the epsilons
epsilons = [0, 4 / 255, 0.05, 0.1, 0.2, 0.35, 0.55, 0.75, 1]

In [29]:
for epsilon in epsilons:
    attacking.attack_model(
        standard_model,
        testSetLoader,
        "PGD",
        attacks["PGD"],
        epsilon=epsilon,
        alpha=(2 / 255),
        iterations=20,
    )

Testing the model under PGD Attack using epsilon = 0, alpha = 0.00784313725490196...


PGD Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 95.25%


Testing the model under PGD Attack using epsilon = 0.01568627450980392, alpha = 0.00784313725490196...


PGD Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 0.28%


Testing the model under PGD Attack using epsilon = 0.05, alpha = 0.00784313725490196...


...

... done! Accuracy: 0.0%


Testing the model under PGD Attack using epsilon = 0.75, alpha = 0.00784313725490196...


PGD Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 0.0%


Testing the model under PGD Attack using epsilon = 1, alpha = 0.00784313725490196...


PGD Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 0.0%




## From here, use existing libraries to simplify usageimport torchattacks


In [30]:
import torchattacks

### DeepFool https://arxiv.org/pdf/1511.04599.pdf

The formula here is complex, so I recommend for the interested reader to go the above link to read the full paper.

In [31]:
deepfool_attack = torchattacks.DeepFool(standard_model)

attacks["DeepFool"] = deepfool_attack

In [32]:
attacking.attack_model(
    standard_model,
    testSetLoader,
    "DeepFool",
    attacks["DeepFool"],
    library=True,
)

Testing the model under DeepFool Attack...


DeepFool Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

KeyboardInterrupt: 

### CW-$L_{2}$ (Carlini & Wagner using $L_{2}$ norm) https://arxiv.org/pdf/1608.04644.pdf

The formula here is complex, so I recommend for the interested reader to go the above link to read the full paper.

In [ ]:
cw_attack = torchattacks.CW(standard_model, c=20)

attacks["CW"] = cw_attack

In [ ]:
attacking.attack_model(
    standard_model,
    testSetLoader,
    "CW",
    attacks["CW"],
    library=True,
)

## Adversarial Training

### FGSM Adversarial Training

In [8]:
fgsm_model = cifar10.adversarial_training(
    trainSetLoader,
    "FGSM",
    attacks["FGSM"],
    load_if_available=True,
    load_path=SAVE_LOAD_ROOT + "/cifar10_fgsm",
    epsilon=0.45,
)

Found already trained model...
... loaded!


In [9]:
clean_test.test_trained_model(fgsm_model, testSetLoader)

Testing the model...


Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 76.41%


In [10]:
# Save the model
torch.save(fgsm_model, SAVE_LOAD_ROOT + "/cifar10_fgsm")

In [11]:
# Several values to use for the epsilons
epsilons = [0, 0.05, 0.1, 0.2, 0.35, 0.55, 0.75, 1]

In [12]:
# Run test for each epsilon
for epsilon in epsilons:
    attacking.attack_model(
        fgsm_model,
        testSetLoader,
        "FGSM",
        attacks["FGSM"],
        epsilon=epsilon,
    )

Testing the model under FGSM Attack using epsilon = 0, alpha = None...


FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 76.41%


Testing the model under FGSM Attack using epsilon = 0.05, alpha = None...


FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 99.75%


Testing the model under FGSM Attack using epsilon = 0.1, alpha = None...


...

... done! Accuracy: 10.0%


Testing the model under FGSM Attack using epsilon = 0.75, alpha = None...


FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 10.0%


Testing the model under FGSM Attack using epsilon = 1, alpha = None...


FGSM Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 10.0%




In [13]:
# Several values to use for the epsilons
epsilons = [0, 4 / 255, 0.05, 0.1, 0.2, 0.35, 0.55, 0.75, 1]

In [14]:
for epsilon in epsilons:
    attacking.attack_model(
        fgsm_model,
        testSetLoader,
        "PGD",
        attacks["PGD"],
        epsilon=epsilon,
        alpha=(2 / 255),
        iterations=7,
    )

Testing the model under PGD Attack using epsilon = 0, alpha = 0.00784313725490196...


PGD Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 76.41%


Testing the model under PGD Attack using epsilon = 0.01568627450980392, alpha = 0.00784313725490196...


PGD Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 0.78%


Testing the model under PGD Attack using epsilon = 0.05, alpha = 0.00784313725490196...


...

... done! Accuracy: 2.32%


Testing the model under PGD Attack using epsilon = 0.75, alpha = 0.00784313725490196...


PGD Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 0.8%


Testing the model under PGD Attack using epsilon = 1, alpha = 0.00784313725490196...


PGD Attack Testing Progress:   0%|          | 0/79 [00:00<?, ?it/s]

... done! Accuracy: 0.0%




### PGD Adversarial Training

In [ ]:
pgd_model = cifar10.adversarial_training(
    trainSetLoader,
    "PGD",
    attacks["PGD"],
    load_if_available=True,
    load_path=SAVE_LOAD_ROOT + "/cifar10_pgd",
    epsilon=0.45,
    alpha=(2 / 255),
    iterations=7,
)

In [ ]:
clean_test.test_trained_model(pgd_model, testSetLoader)

In [ ]:
# Save the model
torch.save(pgd_model, SAVE_LOAD_ROOT + "/cifar10_pgd")

In [ ]:
# Several values to use for the epsilons
epsilons = [0, 0.05, 0.1, 0.2, 0.35, 0.55, 0.75, 1]

In [ ]:
# Run test for each epsilon
for epsilon in epsilons:
    attacking.attack_model(
        pgd_model,
        testSetLoader,
        "FGSM",
        attacks["FGSM"],
        epsilon=epsilon,
    )

In [ ]:
# Several values to use for the epsilons
epsilons = [0, 4 / 255, 0.05, 0.1, 0.2, 0.35, 0.55, 0.75, 1]

In [ ]:
for epsilon in epsilons:
    attacking.attack_model(
        pgd_model,
        testSetLoader,
        "PGD",
        attacks["PGD"],
        epsilon=epsilon,
        alpha=(2 / 255),
        iterations=7,
    )

In [ ]:
deepfool_attack = torchattacks.DeepFool(pgd_model)

attacks["DeepFool"] = deepfool_attack

In [ ]:
attacking.attack_model(
    pgd_model,
    testSetLoader,
    "DeepFool",
    attacks["DeepFool"],
    library=True,
)

In [ ]:
cw_attack = torchattacks.CW(pgd_model, c=20)

attacks["CW"] = cw_attack

In [ ]:
attacking.attack_model(
    pgd_model,
    testSetLoader,
    "CW",
    attacks["CW"],
    library=True,
)

## Interpolated Adversarial Training

In [ ]:
interpolated_pgd_model = cifar10.interpolated_adversarial_training(
    trainSetLoader,
    "PGD",
    attacks["PGD"],
    load_if_available=True,
    load_path=SAVE_LOAD_ROOT + "/cifar10_interpolated_pgd",
    epsilon=0.45,
    alpha=(2 / 255),
    iterations=7,
)

In [ ]:
clean_test.test_trained_model(interpolated_pgd_model, testSetLoader)

In [ ]:
# Save the model
torch.save(interpolated_pgd_model, SAVE_LOAD_ROOT + "/cifar10_interpolated_pgd")

In [ ]:
# Several values to use for the epsilons
epsilons = [0, 0.05, 0.1, 0.2, 0.35, 0.55, 0.75, 1]

In [ ]:
# Run test for each epsilon
for epsilon in epsilons:
    attacking.attack_model(
        interpolated_pgd_model,
        testSetLoader,
        "FGSM",
        attacks["FGSM"],
        epsilon=epsilon,
    )

In [ ]:
# Several values to use for the epsilons
epsilons = [0, 4 / 255, 0.05, 0.1, 0.2, 0.35, 0.55, 0.75, 1]

In [ ]:
for epsilon in epsilons:
    attacking.attack_model(
        interpolated_pgd_model,
        testSetLoader,
        "PGD",
        attacks["PGD"],
        epsilon=epsilon,
        alpha=(2 / 255),
        iterations=7,
    )

## Dual / Triple Adversarial Training

In [ ]:
dual_model = cifar10.dual_adversarial_training(
    trainSetLoader,
    attacks["PGD"],
    attacks["FGSM"],
    load_if_available=True,
    load_path=SAVE_LOAD_ROOT + "/cifar10_dual",
    epsilon=0.45,
    alpha=(2 / 255),
    iterations=7,
)

In [ ]:
clean_test.test_trained_model(dual_model, testSetLoader)

In [ ]:
# Save the model
torch.save(dual_model, SAVE_LOAD_ROOT + "/cifar10_dual")

In [ ]:
# Several values to use for the epsilons
epsilons = [0, 0.05, 0.1, 0.2, 0.35, 0.55, 0.75, 1]

In [ ]:
# Run test for each epsilon
for epsilon in epsilons:
    attacking.attack_model(
        dual_model,
        testSetLoader,
        "FGSM",
        attacks["FGSM"],
        epsilon=epsilon,
    )

In [ ]:
# Several values to use for the epsilons
epsilons = [0, 4 / 255, 0.05, 0.1, 0.2, 0.35, 0.55, 0.75, 1]

In [ ]:
for epsilon in epsilons:
    attacking.attack_model(
        dual_model,
        testSetLoader,
        "PGD",
        attacks["PGD"],
        epsilon=epsilon,
        alpha=(2 / 255),
        iterations=7,
    )

In [ ]:
deepfool_attack = torchattacks.DeepFool(dual_model)

attacks["DeepFool"] = deepfool_attack

In [ ]:
attacking.attack_model(
    dual_model,
    testSetLoader,
    "DeepFool",
    attacks["DeepFool"],
    library=True,
)

In [ ]:
cw_attack = torchattacks.CW(dual_model, c=20)

attacks["CW"] = cw_attack

In [ ]:
attacking.attack_model(
    dual_model,
    testSetLoader,
    "CW",
    attacks["CW"],
    library=True,
)

In [ ]:
triple_model = cifar10.triple_adversarial_training(
    trainSetLoader,
    attacks["PGD"],
    attacks["FGSM"],
    attacks["CW"],
    load_if_available=True,
    load_path=SAVE_LOAD_ROOT + "/cifar10_triple",
    epsilon=0.45,
    alpha=(2 / 255),
    iterations=7,
)

In [ ]:
clean_test.test_trained_model(triple_model, testSetLoader)

In [ ]:
# Save the model
torch.save(triple_model, SAVE_LOAD_ROOT + "/cifar10_triple")

In [ ]:
# Several values to use for the epsilons
epsilons = [0, 0.05, 0.1, 0.2, 0.35, 0.55, 0.75, 1]

In [ ]:
# Run test for each epsilon
for epsilon in epsilons:
    attacking.attack_model(
        triple_model,
        testSetLoader,
        "FGSM",
        attacks["FGSM"],
        epsilon=epsilon,
    )

In [ ]:
# Several values to use for the epsilons
epsilons = [0, 4 / 255, 0.05, 0.1, 0.2, 0.35, 0.55, 0.75, 1]

In [ ]:
for epsilon in epsilons:
    attacking.attack_model(
        triple_model,
        testSetLoader,
        "PGD",
        attacks["PGD"],
        epsilon=epsilon,
        alpha=(2 / 255),
        iterations=7,
    )

In [ ]:
deepfool_attack = torchattacks.DeepFool(triple_model)

attacks["DeepFool"] = deepfool_attack

In [ ]:
attacking.attack_model(
    triple_model,
    testSetLoader,
    "DeepFool",
    attacks["DeepFool"],
    library=True,
)

In [ ]:
cw_attack = torchattacks.CW(triple_model, c=20)

attacks["CW"] = cw_attack

In [ ]:
attacking.attack_model(
    triple_model,
    testSetLoader,
    "CW",
    attacks["CW"],
    library=True,
)

## Jacobian Regularization Training

In [ ]:
jacobian_model = cifar10.jacobian_training(
    trainSetLoader,
    load_if_available=True,
    load_path=SAVE_LOAD_ROOT + "/cifar10_jacobian",
)

In [ ]:
clean_test.test_trained_model(jacobian_model, testSetLoader)

In [ ]:
# Save the model
torch.save(jacobian_model, SAVE_LOAD_ROOT + "/cifar10_jacobian")

In [ ]:
# Several values to use for the epsilons
epsilons = [0, 0.05, 0.1, 0.2, 0.35, 0.55, 0.75, 1]

In [ ]:
# Run test for each epsilon
for epsilon in epsilons:
    attacking.attack_model(
        jacobian_model,
        testSetLoader,
        "FGSM",
        attacks["FGSM"],
        epsilon=epsilon,
    )

In [ ]:
# Several values to use for the epsilons
epsilons = [0, 4 / 255, 0.05, 0.1, 0.2, 0.35, 0.55, 0.75, 1]

In [ ]:
for epsilon in epsilons:
    attacking.attack_model(
        jacobian_model,
        testSetLoader,
        "PGD",
        attacks["PGD"],
        epsilon=epsilon,
        alpha=(2 / 255),
        iterations=7,
    )

In [ ]:
deepfool_attack = torchattacks.DeepFool(jacobian_model)

attacks["DeepFool"] = deepfool_attack

In [ ]:
attacking.attack_model(
    jacobian_model,
    testSetLoader,
    "DeepFool",
    attacks["DeepFool"],
    library=True,
)

In [ ]:
cw_attack = torchattacks.CW(jacobian_model, c=20)

attacks["CW"] = cw_attack

In [ ]:
attacking.attack_model(
    jacobian_model,
    testSetLoader,
    "CW",
    attacks["CW"],
    library=True,
)

## ALP Training

In [ ]:
alp_model = cifar10.ALP_training(
    trainSetLoader,
    "PGD",
    attacks["PGD"],
    load_if_available=True,
    load_path=SAVE_LOAD_ROOT + "/cifar10_alp",
    epsilon=0.45,
    alpha=(2 / 255),
    iterations=7,
)

In [ ]:
clean_test.test_trained_model(alp_model, testSetLoader)

In [ ]:
# Save the model
torch.save(alp_model, SAVE_LOAD_ROOT + "/cifar10_alp")

In [ ]:
# Several values to use for the epsilons
epsilons = [0, 0.05, 0.1, 0.2, 0.35, 0.55, 0.75, 1]

In [ ]:
# Run test for each epsilon
for epsilon in epsilons:
    attacking.attack_model(
        alp_model,
        testSetLoader,
        "FGSM",
        attacks["FGSM"],
        epsilon=epsilon,
    )

In [ ]:
# Several values to use for the epsilons
epsilons = [0, 4 / 255, 0.05, 0.1, 0.2, 0.35, 0.55, 0.75, 1]

In [ ]:
for epsilon in epsilons:
    attacking.attack_model(
        alp_model,
        testSetLoader,
        "PGD",
        attacks["PGD"],
        epsilon=epsilon,
        alpha=(2 / 255),
        iterations=7,
    )

In [ ]:
deepfool_attack = torchattacks.DeepFool(alp_model)

attacks["DeepFool"] = deepfool_attack

In [ ]:
attacking.attack_model(
    alp_model,
    testSetLoader,
    "DeepFool",
    attacks["DeepFool"],
    library=True,
)

In [ ]:
cw_attack = torchattacks.CW(alp_model, c=20)

attacks["CW"] = cw_attack

In [ ]:
attacking.attack_model(
    alp_model,
    testSetLoader,
    "CW",
    attacks["CW"],
    library=True,
)